#### Import required library


In [0]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col,concat, lit
from pyspark.sql.types import IntegerType, DoubleType, BooleanType, DateType
from pyspark.sql.functions import row_number
from pyspark.sql.window import Window


In [0]:

# Start Spark session
spark = SparkSession.builder.appName("TeamAnalysis").getOrCreate()


#### Ideally you load data from ADLS Gen 2 (using cred in Azure key vault) but for test purpose loading it from csv file 
for ex:


configs = {"fs.azure.account.auth.type": "OAuth",
"fs.azure.account.oauth.provider.type": "org.apache.hadoop.fs.azurebfs.oauth2.ClientCredsTokenProvider",
"fs.azure.account.oauth2.client.id": "",
"fs.azure.account.oauth2.client.secret": '',
"fs.azure.account.oauth2.client.endpoint": "https://login.microsoftonline.com/tanent_id/oauth2/token"}


dbutils.fs.mount(
source = "abfss://nfl@nfldatastat.dfs.core.windows.net", # contrainer @ storageacc
mount_point = "/mnt/tokyoolymic",
extra_configs = configs)


In [0]:
# laod the data
df = spark.read.options(inferSchema="true", header="true").csv("/FileStore/tables/team_stats_2003_2023.csv")

###@ Display data , Display Columns, Display Schema

In [0]:
# display data
df.display()
# display columns 
df.columns
# display schema
df.printSchema()

year,team,wins,losses,win_loss_perc,points,points_opp,points_diff,mov,g,total_yards,plays_offense,yds_per_play_offense,turnovers,fumbles_lost,first_down,pass_cmp,pass_att,pass_yds,pass_td,pass_int,pass_net_yds_per_att,pass_fd,rush_att,rush_yds,rush_td,rush_yds_per_att,rush_fd,penalties,penalties_yds,pen_fd,score_pct,turnover_pct,exp_pts_tot,ties
2003,New England Patriots,14,2,0.875,348,238,110,6.9,16,5039,1042,4.8,24,11,294,320,537,3432,23,13,6.0,177,473,1607,9,3.4,91,111,998,26,27.9,11.3,-136.51,null
2003,Miami Dolphins,10,6,0.625,311,261,50,3.1,16,4609,968,4.8,34,15,266,257,450,2792,17,19,5.8,145,487,1817,14,3.7,99,103,913,22,28.1,17.2,-177.92,null
2003,Buffalo Bills,6,10,0.375,243,279,-36,-2.3,16,4348,980,4.4,34,17,268,293,502,2684,11,17,4.9,150,427,1664,13,3.9,96,106,891,22,21.9,17.6,-230.07,null
2003,New York Jets,6,10,0.375,283,299,-16,-1.0,16,4951,936,5.3,20,6,274,312,496,3316,20,14,6.3,181,409,1635,8,4.0,78,69,550,15,32.4,11.8,-107.89,null
2003,Baltimore Ravens,10,6,0.625,391,281,110,6.9,16,4929,1009,4.9,38,19,259,217,415,2255,16,19,4.9,121,552,2674,18,4.8,115,126,970,23,31.8,16.6,-220.5,null
2003,Cincinnati Bengals,8,8,0.5,346,384,-38,-2.4,16,5329,1038,5.1,22,7,313,324,520,3342,26,15,6.0,181,481,1987,12,4.1,101,107,846,31,33.3,11.1,-78.22,null
2003,Pittsburgh Steelers,6,10,0.375,300,327,-27,-1.7,16,4792,1020,4.7,28,11,275,306,532,3304,19,17,5.8,174,446,1488,10,3.3,77,111,1005,24,27.1,13.5,-166.93,null
2003,Cleveland Browns,5,11,0.313,254,322,-68,-4.3,16,4504,961,4.7,33,15,276,313,509,2834,17,18,5.2,153,412,1670,8,4.1,91,98,767,32,26.7,17.2,-191.88,null
2003,Indianapolis Colts,12,4,0.75,447,336,111,6.9,16,5874,1041,5.6,20,10,348,381,569,4179,29,10,7.1,212,453,1695,16,3.7,104,92,662,32,46.3,10.2,-13.96,null
2003,Tennessee Titans,12,4,0.75,435,324,111,6.9,16,5501,1013,5.4,21,12,310,315,502,3878,30,9,7.4,211,486,1623,11,3.3,84,110,887,15,39.4,9.6,-103.79,null


root
 |-- year: integer (nullable = true)
 |-- team: string (nullable = true)
 |-- wins: integer (nullable = true)
 |-- losses: integer (nullable = true)
 |-- win_loss_perc: double (nullable = true)
 |-- points: integer (nullable = true)
 |-- points_opp: integer (nullable = true)
 |-- points_diff: integer (nullable = true)
 |-- mov: double (nullable = true)
 |-- g: integer (nullable = true)
 |-- total_yards: integer (nullable = true)
 |-- plays_offense: integer (nullable = true)
 |-- yds_per_play_offense: double (nullable = true)
 |-- turnovers: integer (nullable = true)
 |-- fumbles_lost: integer (nullable = true)
 |-- first_down: integer (nullable = true)
 |-- pass_cmp: integer (nullable = true)
 |-- pass_att: integer (nullable = true)
 |-- pass_yds: integer (nullable = true)
 |-- pass_td: integer (nullable = true)
 |-- pass_int: integer (nullable = true)
 |-- pass_net_yds_per_att: double (nullable = true)
 |-- pass_fd: integer (nullable = true)
 |-- rush_att: integer (nullable = tru

#### Chnage the datatype if nedded for columns

In [0]:
# convert ties column from int to bool
df = df.withColumn("ties", col("ties").cast(BooleanType()))
df.printSchema()


root
 |-- year: integer (nullable = true)
 |-- team: string (nullable = true)
 |-- wins: integer (nullable = true)
 |-- losses: integer (nullable = true)
 |-- win_loss_perc: double (nullable = true)
 |-- points: integer (nullable = true)
 |-- points_opp: integer (nullable = true)
 |-- points_diff: integer (nullable = true)
 |-- mov: double (nullable = true)
 |-- g: integer (nullable = true)
 |-- total_yards: integer (nullable = true)
 |-- plays_offense: integer (nullable = true)
 |-- yds_per_play_offense: double (nullable = true)
 |-- turnovers: integer (nullable = true)
 |-- fumbles_lost: integer (nullable = true)
 |-- first_down: integer (nullable = true)
 |-- pass_cmp: integer (nullable = true)
 |-- pass_att: integer (nullable = true)
 |-- pass_yds: integer (nullable = true)
 |-- pass_td: integer (nullable = true)
 |-- pass_int: integer (nullable = true)
 |-- pass_net_yds_per_att: double (nullable = true)
 |-- pass_fd: integer (nullable = true)
 |-- rush_att: integer (nullable = tru

#### Filter the data for last 3 years

In [0]:
# filter the data for last 3 years 

years = df.select("year").distinct().orderBy(col("year").desc()).limit(3)
#display(years)
last_3_years = [row["year"] for row in years.collect()]
#display(last_3_years)
# Filter the DataFrame
filtered_df = df.filter(col("year").isin(last_3_years))
filtered_df.display()


year,team,wins,losses,win_loss_perc,points,points_opp,points_diff,mov,g,total_yards,plays_offense,yds_per_play_offense,turnovers,fumbles_lost,first_down,pass_cmp,pass_att,pass_yds,pass_td,pass_int,pass_net_yds_per_att,pass_fd,rush_att,rush_yds,rush_td,rush_yds_per_att,rush_fd,penalties,penalties_yds,pen_fd,score_pct,turnover_pct,exp_pts_tot,ties
2021,Buffalo Bills,11,6,0.647,483,289,194,null,17,6493,1143,5.7,22,6,398,415,655,4284,36,16,6.3,236,461,2209,20,4.8,134,113,980,28,45.2,11.3,216.68,false
2021,New England Patriots,10,7,0.588,462,303,159,null,17,6008,1052,5.7,23,10,362,364,535,3857,24,13,6.9,187,489,2151,24,4.4,139,95,854,36,48.0,12.6,126.97,false
2021,Miami Dolphins,9,8,0.529,341,373,-32,null,17,5219,1097,4.8,26,12,325,404,615,3651,21,14,5.6,206,442,1568,12,3.5,87,105,825,32,30.1,12.4,-11.96,false
2021,New York Jets,4,13,0.235,310,504,-194,null,17,5208,1036,5.0,27,7,310,357,603,3541,20,20,5.4,195,380,1667,14,4.4,87,103,860,28,29.4,14.4,-32.71,false
2021,Cincinnati Bengals,10,7,0.588,460,376,84,null,17,6145,1046,5.9,21,7,337,384,555,4403,36,14,7.2,208,436,1742,16,4.0,96,72,620,33,42.6,10.0,118.5,false
2021,Pittsburgh Steelers,9,7,0.559,343,398,-55,null,17,5361,1113,4.8,20,9,323,425,664,3778,23,11,5.4,204,411,1583,10,3.9,85,106,831,34,34.7,9.5,-14.49,true
2021,Cleveland Browns,8,9,0.471,349,371,-22,null,17,5791,1054,5.5,22,8,342,320,520,3320,21,14,5.8,177,485,2471,20,5.1,138,112,1035,27,32.2,12.4,58.49,false
2021,Baltimore Ravens,8,9,0.471,387,392,-5,null,17,6440,1185,5.4,26,8,394,396,611,3961,21,18,5.9,209,517,2479,18,4.8,159,102,852,26,38.5,13.3,59.67,false
2021,Tennessee Titans,12,5,0.706,419,354,65,null,17,5822,1133,5.1,25,11,362,359,535,3418,22,14,5.9,194,551,2404,23,4.4,134,103,973,34,38.9,13.0,79.19,false
2021,Indianapolis Colts,9,8,0.529,451,365,86,null,17,5901,1052,5.6,19,11,354,324,521,3361,27,8,6.1,167,499,2540,22,5.1,154,76,688,33,43.3,8.9,127.06,false


#### Create unique team_id for each team

In [0]:
lasy_3_yrs_df= filtered_df
#lasy_3_yrs_df.display()

# Step 1: Create unique team_id for each team
window_spec = Window.orderBy("team")
teams_with_id = lasy_3_yrs_df.select("team").distinct().withColumn("team_id", row_number().over(window_spec))

# Step 2: Join back to original DataFrame to retain all columns
df_with_team_id = lasy_3_yrs_df.join(teams_with_id, on="team", how="left").orderBy("team", "year")
df_with_team_id.display()



/databricks/python/lib/python3.12/site-packages/pyspark/sql/connect/expressions.py:1061: UserWarning: WARN WindowExpression: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
  warnings.warn(


team,year,wins,losses,win_loss_perc,points,points_opp,points_diff,mov,g,total_yards,plays_offense,yds_per_play_offense,turnovers,fumbles_lost,first_down,pass_cmp,pass_att,pass_yds,pass_td,pass_int,pass_net_yds_per_att,pass_fd,rush_att,rush_yds,rush_td,rush_yds_per_att,rush_fd,penalties,penalties_yds,pen_fd,score_pct,turnover_pct,exp_pts_tot,ties,team_id
Arizona Cardinals,2021,11,6,0.647,449,366,83,null,17,6352,1126,5.6,15,4,367,415,591,4276,27,11,6.8,214,496,2076,23,4.2,127,114,990,26,44.7,7.8,119.94,false,1
Arizona Cardinals,2022,4,13,0.235,340,449,-109,null,17,5499,1144,4.8,25,8,326,433,664,3626,17,17,5.1,189,434,1873,15,4.3,115,118,916,22,32.1,12.8,-36.27,false,1
Arizona Cardinals,2023,4,13,0.235,330,455,-125,-7.4,17,5509,1068,5.2,18,6,330,355,555,3144,18,12,5.3,173,471,2365,17,5.0,127,101,859,30,35.0,9.4,-2.99,null,1
Atlanta Falcons,2021,7,10,0.412,313,459,-146,null,17,5164,1006,5.1,26,11,306,377,573,3713,20,15,6.1,196,393,1451,11,3.7,75,92,753,35,32.8,14.7,-18.38,false,2
Atlanta Falcons,2022,7,10,0.412,365,386,-21,null,17,5417,1011,5.4,21,12,330,257,415,2699,17,9,6.0,148,559,2718,17,4.9,152,68,594,30,39.5,11.4,73.69,false,2
Atlanta Falcons,2023,7,10,0.412,321,373,-52,-3.1,17,5683,1092,5.2,28,11,326,327,530,3524,17,17,6.2,181,522,2159,14,4.1,121,84,743,24,32.1,14.3,-32.52,null,2
Baltimore Ravens,2021,8,9,0.471,387,392,-5,null,17,6440,1185,5.4,26,8,394,396,611,3961,21,18,5.9,209,517,2479,18,4.8,159,102,852,26,38.5,13.3,59.67,false,3
Baltimore Ravens,2022,10,7,0.588,350,315,35,null,17,5760,1052,5.5,21,8,337,300,488,3040,19,13,5.8,149,526,2720,14,5.2,167,82,678,21,38.9,11.7,48.17,false,3
Baltimore Ravens,2023,13,4,0.765,483,280,203,11.9,17,6296,1076,5.9,19,12,360,328,494,3635,27,7,6.8,180,541,2661,26,4.9,145,102,955,35,43.1,9.6,130.64,null,3
Buffalo Bills,2021,11,6,0.647,483,289,194,null,17,6493,1143,5.7,22,6,398,415,655,4284,36,16,6.3,236,461,2209,20,4.8,134,113,980,28,45.2,11.3,216.68,false,4


#### Creates normalization : make three tables 
#### 1.Season 2.team  3.team_performnace

In [0]:
# group teams in single dataframe (table)
filtered_df= df_with_team_id
# add season id custom column 
df_seson = filtered_df.withColumn("seasonid", concat(lit("season_"), col("year")))
# df_seson.display()
df_seson_year = df_seson.select("seasonid", "year").distinct().orderBy("year")
df_seson_year.display()


df_seson_year.write \
    .mode("overwrite") \
    .format("csv") \
    .save("/FileStore/tables/Season.csv")


yeards_df = [row["year"] for row in years.collect()]
# display(yeards_df)

# Step 1: Get distinct teams and team_id
df_teams = filtered_df.select("team","team_id").distinct().orderBy("team")
df_teams.display()

df_teams.write \
    .mode("overwrite") \
    .format("csv") \
    .save("/FileStore/tables/team.csv")



# List of columns to exclude
excluded_cols = ["team", "season_id", "year"]

# Select all columns except the excluded ones
selected_df = df_seson.select([column for column in df_seson.columns if column not in excluded_cols])
selected_df.display()
selected_df.write \
    .mode("overwrite") \
    .format("csv") \
    .save("/FileStore/tables/team_performnace.csv")




/databricks/python/lib/python3.12/site-packages/pyspark/sql/connect/expressions.py:1061: UserWarning: WARN WindowExpression: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
  warnings.warn(


seasonid,year
season_2021,2021
season_2022,2022
season_2023,2023


team,team_id
Arizona Cardinals,1
Atlanta Falcons,2
Baltimore Ravens,3
Buffalo Bills,4
Carolina Panthers,5
Chicago Bears,6
Cincinnati Bengals,7
Cleveland Browns,8
Dallas Cowboys,9
Denver Broncos,10


wins,losses,win_loss_perc,points,points_opp,points_diff,mov,g,total_yards,plays_offense,yds_per_play_offense,turnovers,fumbles_lost,first_down,pass_cmp,pass_att,pass_yds,pass_td,pass_int,pass_net_yds_per_att,pass_fd,rush_att,rush_yds,rush_td,rush_yds_per_att,rush_fd,penalties,penalties_yds,pen_fd,score_pct,turnover_pct,exp_pts_tot,ties,team_id,seasonid
11,6,0.647,449,366,83,null,17,6352,1126,5.6,15,4,367,415,591,4276,27,11,6.8,214,496,2076,23,4.2,127,114,990,26,44.7,7.8,119.94,false,1,season_2021
4,13,0.235,340,449,-109,null,17,5499,1144,4.8,25,8,326,433,664,3626,17,17,5.1,189,434,1873,15,4.3,115,118,916,22,32.1,12.8,-36.27,false,1,season_2022
4,13,0.235,330,455,-125,-7.4,17,5509,1068,5.2,18,6,330,355,555,3144,18,12,5.3,173,471,2365,17,5.0,127,101,859,30,35.0,9.4,-2.99,null,1,season_2023
7,10,0.412,313,459,-146,null,17,5164,1006,5.1,26,11,306,377,573,3713,20,15,6.1,196,393,1451,11,3.7,75,92,753,35,32.8,14.7,-18.38,false,2,season_2021
7,10,0.412,365,386,-21,null,17,5417,1011,5.4,21,12,330,257,415,2699,17,9,6.0,148,559,2718,17,4.9,152,68,594,30,39.5,11.4,73.69,false,2,season_2022
7,10,0.412,321,373,-52,-3.1,17,5683,1092,5.2,28,11,326,327,530,3524,17,17,6.2,181,522,2159,14,4.1,121,84,743,24,32.1,14.3,-32.52,null,2,season_2023
8,9,0.471,387,392,-5,null,17,6440,1185,5.4,26,8,394,396,611,3961,21,18,5.9,209,517,2479,18,4.8,159,102,852,26,38.5,13.3,59.67,false,3,season_2021
10,7,0.588,350,315,35,null,17,5760,1052,5.5,21,8,337,300,488,3040,19,13,5.8,149,526,2720,14,5.2,167,82,678,21,38.9,11.7,48.17,false,3,season_2022
13,4,0.765,483,280,203,11.9,17,6296,1076,5.9,19,12,360,328,494,3635,27,7,6.8,180,541,2661,26,4.9,145,102,955,35,43.1,9.6,130.64,null,3,season_2023
11,6,0.647,483,289,194,null,17,6493,1143,5.7,22,6,398,415,655,4284,36,16,6.3,236,461,2209,20,4.8,134,113,980,28,45.2,11.3,216.68,false,4,season_2021


#### Set Connection to access your ADLS from Databrick (This is for test but In real envirment read this from Azure key vault)